## Big Query SQL queries used in this project

Here we're collecting lists of page paths and content ids, for which there have been more than 5 page hits in the last 2 weeks.

In [ ]:
# import packages
import pandas as pd
import pandas_gbq

# extract data from BigQuery
project_id = "govuk-bigquery-analytics"

# document types to ignore i.e. pages not relevant to WUJs
ignore = (
    "authored_article",
    "news_article",
    "news_story",
    "press_release",
    "world_news_story",
    "utaac_decision",
    "speech",
    "case_study",
    "raib_report",
    "asylum_support_decision",
    "policy_paper",
    "corporate_report",
    "written_statement",
    "consultation_outcome",
    "closed_consultation",
    "maib_report",
    "person",
    "correspondence",
    "employment_tribunal_decision",
    "employment_appeal_tribunal_decision",
    "tax_tribunal_decision",
    "ministerial_role",
    "residential_property_tribunal_decision",
    "cma_case",
)

### By page hits per page path

In [ ]:
# get page paths with more than 5 page hits in the last 2 weeks,
# ignoring irrelevant pages
sql = """
DECLARE start_date STRING DEFAULT "20211018";
DECLARE end_date STRING DEFAULT "20211101";

WITH sessions AS (
SELECT DISTINCT
    CONCAT(fullVisitorId, "-", CAST(visitId AS STRING)) AS sessionId,
    REGEXP_REPLACE(hits.page.pagePath, r'[?#].*', '') AS pagePath
FROM `govuk-bigquery-analytics.87773428.ga_sessions_*`
CROSS JOIN UNNEST(hits) AS hits
WHERE
    _TABLE_SUFFIX BETWEEN start_date AND end_date
    AND hits.page.pagePath NOT LIKE "/print%"
    AND hits.type = 'PAGE'
    AND (SELECT value FROM hits.customDimensions WHERE index = 2) NOT IN {} 
)

SELECT 
    pagePath,
    COUNT(sessionId) AS noOfSessions 
FROM sessions
GROUP BY pagePath
HAVING noOfSessions >= 5
ORDER BY noOfSessions
""".format(
    ignore
)

# create Data Frame
page_paths = pandas_gbq.read_gbq(sql, project_id=project_id)
page_paths.to_csv("page_paths.csv", index=False)

### By page hits per content ID

In [ ]:
# get content ids with more than 5 page hits in the last 2 weeks,
# ignoring irrelevant pages
sql = """
DECLARE start_date STRING DEFAULT "20211018";
DECLARE end_date STRING DEFAULT "20211101";

WITH sessions AS (
SELECT DISTINCT
    CONCAT(fullVisitorId, "-", CAST(visitId AS STRING)) AS sessionId,
    (SELECT value FROM hits.customDimensions WHERE index = 4) AS contentId
FROM `govuk-bigquery-analytics.87773428.ga_sessions_*`
CROSS JOIN UNNEST(hits) AS hits
WHERE
    _TABLE_SUFFIX BETWEEN start_date AND end_date
    AND hits.page.pagePath NOT LIKE "/print%"
    AND hits.type = 'PAGE'
    AND (SELECT value FROM hits.customDimensions WHERE index = 2) NOT IN {}
)

SELECT 
    contentID,
    COUNT(sessionId) AS noOfSessions 
FROM sessions
GROUP BY contentID
HAVING noOfSessions >= 5
""".format(
    ignore
)

# create Data Frame
content_ids = pandas_gbq.read_gbq(sql, project_id=project_id)
content_ids.to_csv("content_ids.csv", index=False)

### Get SaB pages

In [ ]:
# get all the SaB pages
sql = """
DECLARE start_date STRING DEFAULT "20211018";
DECLARE end_date STRING DEFAULT "20211101";

SELECT DISTINCT
    REGEXP_REPLACE(hits.page.pagePath, r'[?#].*', '') AS pagePath
FROM `govuk-bigquery-analytics.87773428.ga_sessions_*`
CROSS JOIN UNNEST(hits) AS hits
WHERE
    _TABLE_SUFFIX BETWEEN start_date AND end_date
    AND hits.page.pagePath NOT LIKE "/print%"
    AND hits.type = 'PAGE'
    AND REGEXP_REPLACE(hits.page.pagePath, r'[?#].*', '') IN (SELECT string_field_2 FROM `govuk-bigquery-analytics.banner_intervention.sabpages`)
"""

# create Data Frame
sab_pages = pandas_gbq.read_gbq(sql, project_id=project_id)
sab_pages.to_csv("sab_pages.csv", index=False)